In [1]:
import tarfile
import re
import tensorflow as tf

In [34]:
import bz2
import numpy as np

In [36]:
import attrdict

In [3]:
class ImbdReview:
    TOKEN = re.compile(r'[A-Za-z]+|[!?.:,()]')
    
    def __init__(self, cache_dir, file_path):
        self._cache_dir = cache_dir
        self._file_path = file_path
        
    def __iter__(self):
        with tarfile.open(self._file_path) as archive:
            for filename in archive.getnames():
                if filename.startswith('aclImdb/train/pos/'):
                    yield self._read(archive, filename), True
                elif filename.startswith('aclImdb/train/neg/'):
                    yield self._read(archive, filename), False
                    
    def _read(self, archive, filename):
        with archive.extractfile(filename) as file_:
            data = file_.read().decode('utf-8')
            data = type(self).TOKEN.findall(data)
            data = [x.lower() for x in data]
            return data
        

In [35]:
class Embedding:
    
    def __init__(self,vocabulary_path, embedding_path, length):
        self._embedding = np.load(embedding_path)
        with bz2.open(vocabulary_path,'rt') as file_:
            self._vocabulary = {word.strip(): i for i,word in enumerate(file_)}
        self._length = length
        
    def __call__(self, sequence):
        data=np.zeros([self._length, self._embedding.shape[1]])
        indices = [self._vocabulary.get(x,0) for x in sequence]
        embedded = self._embedding[indices]
        data[:len(sequence)] = embedded
        return data
    
    def dimensions(self):
        return self._embedding.shape[1]
    
    

In [37]:
class SequenceClassificationModel:
    
    def __init__(self,data, target, params):
        self.data=data
        self.target=target
        self.params=params
        self.prediction
        self.cost
        self.error
        self.optimize
        
    def length(self):
        used = tf.sign(tf.reduce_max(tf.abs(self.data), axis=2))
        length = tf.reduce_sum(used, axis=1)
        length = tf.cast(length, tf.int32)
        return length
    
    def prediction(self):
        output, _ = tf.nn.dynamic_rnn(self.params.rnn_cell(self.params.rnn_hidden), self.data, dtype=tf.float32, sequence_length=self.length)
        
        last = self._last_relevant(output, self.length)
        num_classes = int(self.target.get_shape()[1])
        weight = tf.Variable(tf.truncated_normal([self.params.rnn_hidden, num_classes],stddev=0.1))
        bias = tf.Variable(tf.constant(0.1,[num_classes]))
        prediction = tf.nn.softmax(tf.matmul(last, weight), bias)
        return prediction
    
    # flatten last from sequences x time_steps x word_vectors 
    def _last_relevant(output, length):
        batch_size = tf.shape(output)[0]
        max_length = int(output.get_shape()[1])
        output_size = int(output.get_shape()[2])
        index = tf.range(0,batch_size) * max_length + (length-1)
        flat = tf.reshape(output, [-1, output_size])
        relevant = tf.gather(flat, index)
        return relevant
    
    def cost(self):
        cross_entropy = -tf.reduce_sum(self.target * tf.log(self.prediction))
        return cross_entropy
    
    def optimize(self):
        gradient = self.params.optimizer.compute_gradient(self.cost)
        # modify the gradients
        if self.params.gradient_clipping:
            limit = self.params.gradient_clipping
            gradient = [(tf.clip_by_value(g, -limit, limit),v) if g is not None else (None, v) for g,v in gradient]
            
            optimize = self.params.optimizer.apply_gradients(gradient)
            
        return optimize
    
    def error(self):
        mistakes = tf.not_equal(tf.argmax(self.target,1),tf.argmax(self.prediction,1))
        return tf.reduce_mean(tf.cast(mistakes), tf.float32)
    
    

In [38]:
def preprocess_batched(iterator, length, embedding, batch_size):
    iterator = iter(iterator)
    while True:
        data = np.zeros((batch_size, length, embedding.dimensions))
        target = np.zeros((batch_size, 2))
        for index in range(batch_size):
            text, label = next(iterator)
            data[index] = embedding(text)
            target[index] = [1, 0] if label else [0, 1]
        yield data, target

In [41]:
params = attrdict.AttrDict(
rnn_cell=tf.contrib.rnn.GRUCell,
rnn_hidden=300,
optimizer=tf.train.RMSPropOptimizer(0.002),
batch_size=20,
)

In [43]:
reviews = ImbdReview(cache_dir='ImdbReview',file_path='aclImdb.tar.gz')

In [45]:
length = max(len(x[0]) for x in reviews)

In [48]:
embedding = Embedding('ImdbReview/vocabulary.bz2','ImdbReview/embedding.npy', length)

FileNotFoundError: [Errno 2] No such file or directory: 'ImdbReview/embedding.npy'

In [47]:

batches = preprocess_batched(reviews, length, embedding, params.batch_size)
data = tf.placeholder(tf.float32, [None, length, embedding.dimensions])
target = tf.placeholder(tf.float32, [None, 2])
model = SequenceClassificationModel(data, target, params)
sess = tf.Session()
sess.run(tf.initialize_all_variables())
for index, batch in enumerate(batches):
feed = {data: batch[0], target: batch[1]}
error, _ = sess.run([model.error, model.optimize], feed)
print('{}: {:3.1f}%'.format(index + 1, 100 * error))

IndentationError: expected an indented block (<ipython-input-47-64e4f4178bc0>, line 9)

In [46]:
length

2758